In [ ]:
# Load preprocessed data
# Load the regression data (for the prediction)
# Calculate interception behaviorally
# Look at prediction of prey position in the future

In [ ]:
# imports
import os
import sys
sys.path.insert(0, os.path.abspath(r'D:\Code Repos\prey_capture'))

import panel as pn
import holoviews as hv
from holoviews import opts, dim
hv.extension('bokeh')
from bokeh.resources import INLINE

import numpy as np
import pandas as pd
import functions_bondjango as bd
import functions_plotting as fp
import paths
import processing_parameters
import importlib


In [ ]:
importlib.reload(fp)
# set up the figure theme
fp.set_theme()

In [ ]:
importlib.reload(processing_parameters)
# get the search query
search_string = processing_parameters.search_string

# query the database for data to plot
data_all = bd.query_database('analyzed_data', search_string)

input_path = [el['analysis_path'] for el in data_all if 'preproc' in el['slug']]

data_list = []
meta_list = []

for idx, el in enumerate(input_path):
    # get the trial timestamp (for frame calculations)
    time_stamp = int(''.join(os.path.basename(el).split('_')[3:6]))
    # also get the trial time signature, used as unique ID
    # time_signature = int(''.join(os.path.basename(el).split('_')[0:6]))

    try:
        temp_data = pd.read_hdf(el, 'matched_calcium')
        # temp_data['trial_id'] = time_signature
        temp_data['id'] = data_all[idx]['id']

        meta_list.append([data_all[idx][el1] for el1 in processing_parameters.meta_fields])
        # try to load the motifs and latents
        try:
            latents = pd.read_hdf(el, 'latents')
            motifs = pd.read_hdf(el, 'motifs')
            egocentric_coords = pd.read_hdf(el, 'egocentric_coord')
            egocentric_coords = egocentric_coords.loc[:, ['cricket_0_x', 'cricket_0_y']]
            egocentric_coords = egocentric_coords.rename(columns={'cricket_0_x': 'ego_cricket_x',
                                                                  'cricket_0_y': 'ego_cricket_y'})
            # determine the delta size for padding
            delta_frames = temp_data.shape[0] - latents.shape[0]
            # pad latents due to the VAME calculation window
            latent_padding = pd.DataFrame(np.zeros((int(delta_frames / 2), len(latents.columns))) * np.nan,
                                          columns=latents.columns)
            motif_padding = pd.DataFrame(np.zeros((int(delta_frames / 2), len(motifs.columns))) * np.nan,
                                         columns=motifs.columns)
            # pad them with nan at the edges (due to VAME excluding the edges
            latents = pd.concat([latent_padding, latents, latent_padding], axis=0).reset_index(drop=True)
            motifs = pd.concat([motif_padding, motifs, motif_padding], axis=0).reset_index(drop=True)
            # concatenate with the main data
            temp_data = pd.concat([temp_data, egocentric_coords, latents, motifs], axis=1)
        except KeyError:
            print(f'No latents in file {el}')
        data_list.append(temp_data)
    except KeyError:
        continue

In [ ]:
# Predict a given variable in the future from the neural activity

# separate train and test set (without shuffling)
                calcium_train, calcium_test, parameter_train, parameter_test = \
                    mod.train_test_split(calcium_data_working, parameter_working, test_size=0.2, shuffle=False)
                # scale the calcium data
                calcium_scaler = preprocessing.StandardScaler().fit(calcium_train)
                calcium_train = calcium_scaler.transform(calcium_train)
                calcium_test = calcium_scaler.transform(calcium_test)
                calcium_data_working = calcium_scaler.transform(calcium_data_working)
                # create the linear regressor
                linear = lin.TweedieRegressor(alpha=0.01, max_iter=5000, fit_intercept=False, power=0)
                # train the regressor
                linear.fit(calcium_train, parameter_train)

In [ ]:
# Detect interception courses from mouse to prey

# get the target trial
current_trial = data_list[0]

# define the variables to plot
plot_variables = ['cricket_0_mouse_distance', 'cricket_0_delta_heading', 'mouse_heading', 'cricket_0_heading']

# allocate a plot list
plot_list = []
# for all the variables
for feature in plot_variables:
#     plot = hv.Curve((current_trial['time_vector'], current_trial[feature]), kdims='Time', vdims=feature)
    plot = hv.Curve(current_trial, kdims='time_vector', vdims=feature)
    plot.opts(width=1200, height=200)
    
    plot_list.append(plot)

# calculate the cb angle
cb_angle = ideal_angle(current_trial['mouse_speed'], current_trial['cricket_0_speed'], current_trial['cricket_0_heading'])
plot = hv.Curve((current_trial['time_vector'], cb_angle))
plot.opts(width=1200, height=200)
plot_list.append(plot)

plot = hv.Curve((current_trial['time_vector'], current_trial['cricket_0_delta_heading'] - cb_angle))
plot.opts(width=1200, height=200)
plot_list.append(plot)

hv.Layout(plot_list).cols(1).opts(shared_axes=False)
    

In [ ]:
# plot the trajectories of mouse and prey

mouse = hv.Scatter(current_trial, kdims='mouse_x', vdims=['mouse_y', 'time_vector'])
mouse_line = hv.Curve(current_trial, kdims='mouse_x', vdims='mouse_y')
mouse.opts(height=1000, width=1000, color='time_vector', cmap='Spectral', size=10)
mouse_line.opts(color='black')
prey_line = hv.Curve(current_trial, kdims='cricket_0_x', vdims='cricket_0_y')
prey = hv.Scatter(current_trial, kdims='cricket_0_x', vdims=['cricket_0_y', 'time_vector'])
prey.opts(size=10)
prey_line.opts(color='blue')
mouse.opts(height=1000, width=1000, color='time_vector')

mouse_line*mouse*prey_line*prey

In [ ]:
# rough interception course calculation

# def interception(mouse_position, prey_position, mouse_speed, prey_speed, mouse_angle, prey_angle):
#     """Calculate the interception point of mouse and prey assuming constant speed and direction, taken from 
#     http://jaran.de/goodbits/2011/07/17/calculating-an-intercept-course-to-a-target-with-constant-direction-and-velocity-in-a-2-dimensional-plane/"""
    
#     # get the delta initial position
#     o = prey_position - mouse_position
#     # get the constants used in the formula
#     h1 = 
#     return
def ideal_angle(mouse_speed, cricket_speed, cricket_heading):
    """Calculate the ideal angle for interception according to Ghose et al. 2006"""
    angle_out = np.rad2deg(np.arcsin(cricket_speed*np.sin(np.deg2rad(cricket_heading))/mouse_speed))
    return angle_out